# Store Financial Market Data on your computer

In this chapter we present the following:
1. Storing data on disk in CSV format
2. Storing data on disk with SQLite
3. Storing data in a networked Postgres database
4. Storing data in ultra-fast HDF5 format

### Storing data on disk in CSV format

In [1]:
# apart from ease of use in Python and its libraries / used in Tabelau, PowerBI, prop systems
# since pandas supports writing data to CSV, there are no special libraries required
import pandas as pd 
from openbb import obb
obb.user.preferences.output_type = "dataframe"

In [2]:
# func to download data, manipulate the results, and return pandas dataframe
def get_stock_data(symbol, start_date=None, end_date=None):
    data = obb.equity.price.historical(
    symbol,
    start_date = start_date,
    end_date = end_date,
    provider="yfinance"
    )
    data.reset_index(inplace=True)
    data['symbol'] = symbol
    return data

In [3]:
# func to save a range of data as a CSV file
def save_data_range(symbol, start_date=None, end_date=None):
    data = get_stock_data(symbol, start_date, end_date)
    data.to_csv(
    f"{symbol}.gz",
    compression="gzip",
    index=False
    )

In [4]:
# implementation of a func that reads a csv file and returns a dataframe
def get_data(symbol):
    return pd.read_csv(
        f"{symbol}.gz",
        compression="gzip",
        index_col="date",
        usecols=[
            "date",
            "open",
            "high",
            "low",
            "close",
            "volume",
            "symbol"
] )

In [5]:
# save data as CSV
save_data_range("PLTR")

### Storing data on disk with SQLite

In [7]:
# a bridge between the simplicity of flat files and robustness of relational databases
# in this directory I will create script called market_data_sqlite
# "exchange_calendars" - python package for quering calendars for trading days
!pip install exchange_calendars --quiet

## look for market_data_sqlite script in the directory